In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold

# Raw Data Loading
train = pd.read_csv('./data/digit-recognizer/train.csv')
test = pd.read_csv('./data/digit-recognizer/test.csv')

# # Data Split
# # 7:3 비율로 train과 test 분리
# # x_data_test, t_data_test는 맨끝에서 모델의 최종 Accuracy를 측정할 때 딱 한번 사용
# x_data_train, x_data_test, t_data_train, t_data_test = \
# train_test_split(df[['height','weight']], df['label'], test_size=0.3, random_state=0)

# # Normalization
# scaler = MinMaxScaler()
# scaler.fit(x_data_train)  # scaling을 하기 위한 정보를 scaler에 setting

# x_data_train_norm = scaler.transform(x_data_train)
# x_data_test_norm = scaler.transform(x_data_test)

# # Tensorflow 구현
# # 해당 데이터는 Multinomial이기 때문에 One-Hot Encoding일 이용해 데이터 변환 필요
# # 0 => 1 0 0
# # 1 => 0 1 0
# # 2 => 0 0 1
# sess = tf.Session()
# t_data_train_onehot = sess.run(tf.one_hot(t_data_train, depth=3))
# t_data_test_onehot = sess.run(tf.one_hot(t_data_test, depth=3))

# # Placeholder
# X = tf.placeholder(shape=[None,2], dtype=tf.float32)
# T = tf.placeholder(shape=[None,3], dtype=tf.float32)

# # Weight & bias
# W = tf.Variable(tf.random.normal([2,3]), name='weight')
# b = tf.Variable(tf.random.normal([3]), name='bias')

# # Hypothesis
# logit = tf.matmul(X,W) + b
# H = tf.nn.softmax(logit)

# # Loss Fucntion
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=T))

# # Train
# train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

# # 반복학습
# # parameter
# num_of_epoch = 1000
# batch_size = 100     # 한번에 학습할 x_data와 t_data의 행의 수

# def run_train(sess, train_x, train_t):
#     print('======= 학습시작 =======')
#     sess.run(tf.global_variables_initializer())
#     total_batch = int(train_x.shape[0] / batch_size)
    
#     for step in range(num_of_epoch):
#         for i in range(total_batch):
#             batch_x = train_x[i*batch_size:(i+1)*batch_size]
#             batch_t = train_t[i*batch_size:(i+1)*batch_size]
#             _, loss_val = sess.run([train,loss], feed_dict={X : batch_x, T : batch_t})

#         if step % 100 == 0 :
#             print('Loss : {}'.format(loss_val))
#     print('======= 학습종료 =======')
    
# # Evaluation(Accuracy)
# predict = tf.argmax(H,1)  # 1 => axis를 의미
# correct = tf.equal(predict, tf.argmax(T,1))
# accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

# # Cross Validation
# cv = 5        # KFold의 K
# results = []   # 각 set에서 구한 accuracy를 집어넣을 list
# kf = KFold(n_splits=cv, shuffle=True)

# for training_idx, validation_idx in kf.split(x_data_train_norm):
#     train_x = x_data_train_norm[training_idx]
#     train_t = t_data_train_onehot[training_idx]
#     valid_x = x_data_train_norm[validation_idx]
#     valid_t = t_data_train_onehot[validation_idx]

#     run_train(sess, train_x, train_t)
#     results.append(sess.run(accuracy, feed_dict={X: valid_x, T : valid_t}))

# print('Cross Validation 결과 : {}'.format(np.mean(results)))

In [ ]:
# Data Split
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(train.iloc[:,1:], train['label'], test_size=0.3, random_state=0)

In [ ]:
# Normalization
scaler = MinMaxScaler()
scaler.fit(x_data_train)  # scaling을 하기 위한 정보를 scaler에 setting

x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

In [ ]:
sess = tf.Session()
t_data_train_onehot = sess.run(tf.one_hot(t_data_train, depth=10))
t_data_test_onehot = sess.run(tf.one_hot(t_data_test, depth=10))

In [ ]:
# Placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
T = tf.placeholder(shape=[None,10], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random.normal([784,10]), name='weight')
b = tf.Variable(tf.random.normal([10]), name='bias')

# Hypothesis
logit = tf.matmul(X,W) + b
H = tf.nn.softmax(logit)

# Loss Fucntion
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=T))

# Train
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

# 반복학습
# parameter
num_of_epoch = 1000
batch_size = 100     # 한번에 학습할 x_data와 t_data의 행의 수

def run_train(sess, train_x, train_t):
    print('======= 학습시작 =======')
    sess.run(tf.global_variables_initializer())
    total_batch = int(train_x.shape[0] / batch_size)
    
    for step in range(num_of_epoch):
        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            _, loss_val = sess.run([train,loss], feed_dict={X : batch_x, T : batch_t})

        if step % 100 == 0 :
            print('Loss : {}'.format(loss_val))
    print('======= 학습종료 =======')

In [ ]:
# Evaluation(Accuracy)
predict = tf.argmax(H,1)  # 1 => axis를 의미
correct = tf.equal(predict, tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

# Cross Validation
cv = 5        # KFold의 K
results = []   # 각 set에서 구한 accuracy를 집어넣을 list
kf = KFold(n_splits=cv, shuffle=True)

for training_idx, validation_idx in kf.split(x_data_train_norm):
    train_x = x_data_train_norm[training_idx]
    train_t = t_data_train_onehot[training_idx]
    valid_x = x_data_train_norm[validation_idx]
    valid_t = t_data_train_onehot[validation_idx]

    run_train(sess, train_x, train_t)
    results.append(sess.run(accuracy, feed_dict={X: valid_x, T : valid_t}))

print('Cross Validation 결과 : {}'.format(np.mean(results)))

In [ ]:
# 학습진행
run_train(sess, x_data_train_norm, t_data_train_onehot)

# Accuracy 측정
result = sess.run(accuracy, feed_dict={X: x_data_test_norm, T : t_data_test_onehot})
print('최종 Accuracy : {}'.format(result))

# Prediction
test_scaled = scaler.transform(test)

result = sess.run(H, feed_dict={X : test_scaled})

In [ ]:
print(np.argmax(result[0]))
print(result.shape)

In [ ]:
df = pd.DataFrame()
imageid = list(range(1,result.shape[0]+1))
label = []

for i in range(result.shape[0]):
    label.append(np.argmax(result[i]))
    
df['ImageId'] = imageid
df['Label'] = label

df.to_csv('./submission.csv', index=False)

In [ ]:
### 2020/10/16

In [ ]:
%reset

# Tensorflow 1.15버전
# multinomial classification으로 MNIST 구현

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler        # Normalization
from sklearn.model_selection import train_test_split  # train, test 분리
from sklearn.metrics import classification_report

# Raw Data Loading
df = pd.read_csv('./data/mnist/train.csv')

##### 결측치와 이상치는 없음 #####

##### 이미지 확인 #####
img_data = df.drop('label', axis=1, inplace=False).values

fig = plt.figure()
fig_arr = []

for n in range(10):
    fig_arr.append(fig.add_subplot(2,5,n+1))
    fig_arr[n].imshow(img_data[n].reshape(28,28), cmap='Greys')

plt.tight_layout()
plt.show()


# Data Split
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df.drop('label', axis=1, inplace=False), df['label'], test_size=0.3, random_state=0)

# Min-Max Normalization
scaler = MinMaxScaler()   # scaler = StandardScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train
del x_data_test

##### Tensorflow implementation #####

sess = tf.Session()

t_data_train_onehot = sess.run(tf.one_hot(t_data_train,depth=10))
t_data_test_onehot = sess.run(tf.one_hot(t_data_test,depth=10))

# Placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
T = tf.placeholder(shape=[None,10], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random.normal([784,10]), name='weight')
b = tf.Variable(tf.random.normal([10]), name='bias')

# Hypothesis
logit = tf.matmul(X,W) + b
H = tf.nn.softmax(logit)   # softmax activation function

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, 
                                                                 labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

# parameter
num_of_epoch = 200
batch_size = 100


# 학습
def run_train(sess, train_x, train_t):
    print('### Starting Training ###')
    # 초기화
    sess.run(tf.global_variables_initializer())
    
    for step in range(num_of_epoch):
        total_batch = int(train_x.shape[0] / batch_size)

        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            _, loss_val = sess.run([train,loss], feed_dict={X:batch_x, T:batch_t})

        if step % 20 == 0:
            print('Loss : {}'.format(loss_val))
    print('### End Training ###')

    
# Accuracy
predict = tf.argmax(H,1)

   
# Testing
run_train(sess,x_data_train_norm,t_data_train_onehot) # 학습
print('### Test Set으로 Accuracy 측정 ###')
result = sess.run(predict, feed_dict={X:x_data_test_norm})
print(classification_report(t_data_test,result.ravel()))

In [ ]:
%reset

# Tensorflow 1.15버전
# DNN(초기화, activation function, dropout처리 안함)으로 MNIST 구현

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler        # Normalization
from sklearn.model_selection import train_test_split  # train, test 분리
from sklearn.metrics import classification_report

# Raw Data Loading
df = pd.read_csv('./data/digit-recognizer/train.csv')
display(df.head(), df.shape)

##### 결측치와 이상치는 없음 #####

# Data Split
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df.drop('label', axis=1, inplace=False), df['label'], test_size=0.3, random_state=0)
# test_size : test set의 비율 (0.3 => 30%)
# random_state : split할 때 랜덤하게 split하게 되는데 이를 일정하게 고정(seed의 개념)

# Min-Max Normalization
scaler = MinMaxScaler()   # scaler = StandardScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train
del x_data_test

##### Tensorflow implementation #####

sess = tf.Session()

t_data_train_onehot = sess.run(tf.one_hot(t_data_train,depth=10))
t_data_test_onehot = sess.run(tf.one_hot(t_data_test,depth=10))

# del t_data_train
# del t_data_test

# Placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
T = tf.placeholder(shape=[None,10], dtype=tf.float32)

# Weight & bias
W2 = tf.Variable(tf.random.normal([784,256]), name='weight2')
b2 = tf.Variable(tf.random.normal([256]), name='bias2')
layer2 = tf.sigmoid(tf.matmul(X,W2) + b2)

W3 = tf.Variable(tf.random.normal([256,128]), name='weight3')
b3 = tf.Variable(tf.random.normal([128]), name='bias3')
layer3 = tf.sigmoid(tf.matmul(layer2,W3) + b3)

W4 = tf.Variable(tf.random.normal([128,10]), name='weight4')
b4 = tf.Variable(tf.random.normal([10]), name='bias4')

# Hypothesis
logit = tf.matmul(layer3,W4) + b4
H = tf.nn.softmax(logit)   # softmax activation function

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, 
                                                                 labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

# parameter
num_of_epoch = 500
batch_size = 100


# 학습
def run_train(sess, train_x, train_t):
    print('### Starting Training ###')
    # 초기화
    sess.run(tf.global_variables_initializer())
    
    for step in range(num_of_epoch):
        total_batch = int(train_x.shape[0] / batch_size)

        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            _, loss_val = sess.run([train,loss], feed_dict={X:batch_x, T:batch_t})

        if step % 50 == 0:
            print('Loss : {}'.format(loss_val))
    print('### End Training ###')

    
# Accuracy
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

# Testing
run_train(sess,x_data_train_norm,t_data_train_onehot) # 학습
print('### Test Set으로 Accuracy 측정 ###')
result = sess.run(predict, feed_dict={X:x_data_test_norm})
print(classification_report(t_data_test,result.ravel()))
# Test Set 정확도 : 0.93

In [1]:
%reset

# Tensorflow 1.15버전
# DNN(dropout처리 안함)으로 MNIST 구현

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler        # Normalization
from sklearn.model_selection import train_test_split  # train, test 분리
from sklearn.metrics import classification_report

# Raw Data Loading
df = pd.read_csv('./data/digit-recognizer/train.csv')
display(df.head(), df.shape)

##### 결측치와 이상치는 없음 #####

# Data Split
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df.drop('label', axis=1, inplace=False), df['label'], test_size=0.3, random_state=0)
# test_size : test set의 비율 (0.3 => 30%)
# random_state : split할 때 랜덤하게 split하게 되는데 이를 일정하게 고정(seed의 개념)

# Min-Max Normalization
scaler = MinMaxScaler()   # scaler = StandardScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train
del x_data_test

##### Tensorflow implementation #####

sess = tf.Session()

t_data_train_onehot = sess.run(tf.one_hot(t_data_train,depth=10))
t_data_test_onehot = sess.run(tf.one_hot(t_data_test,depth=10))

# del t_data_train
# del t_data_test

# Placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
T = tf.placeholder(shape=[None,10], dtype=tf.float32)

# Weight & bias
# He's 초기화
W2 = tf.get_variable('w2', shape=[784,256], initializer=tf.contrib.layers.variance_scaling_initializer())
b2 = tf.Variable(tf.random.normal([256]), name='bias2')
layer2 = tf.nn.relu(tf.matmul(X,W2) + b2)

W3 = tf.get_variable('w3', shape=[256,128], initializer=tf.contrib.layers.variance_scaling_initializer())
b3 = tf.Variable(tf.random.normal([128]), name='bias3')
layer3 = tf.nn.relu(tf.matmul(layer2,W3) + b3)

W4 = tf.get_variable('w4', shape=[128,10], initializer=tf.contrib.layers.variance_scaling_initializer())
b4 = tf.Variable(tf.random.normal([10]), name='bias4')

# Hypothesis
logit = tf.matmul(layer3,W4) + b4
H = tf.nn.softmax(logit)   # softmax activation function

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, 
                                                                 labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

# parameter
num_of_epoch = 500
batch_size = 100


# 학습
def run_train(sess, train_x, train_t):
    print('### Starting Training ###')
    # 초기화
    sess.run(tf.global_variables_initializer())
    
    for step in range(num_of_epoch):
        total_batch = int(train_x.shape[0] / batch_size)

        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            _, loss_val = sess.run([train,loss], feed_dict={X:batch_x, T:batch_t})

        if step % 50 == 0:
            print('Loss : {}'.format(loss_val))
    print('### End Training ###')

    
# Accuracy
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

# Testing
run_train(sess,x_data_train_norm,t_data_train_onehot) # 학습
print('### Test Set으로 Accuracy 측정 ###')
result = sess.run(predict, feed_dict={X:x_data_test_norm})
print(classification_report(t_data_test,result.ravel()))

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(42000, 785)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

### Starting Training ###
Loss : 0.2679951786994934
Loss : 0.0017695678398013115
Loss : 0.0005366549012251198
Loss : 0.00030197089654393494
Loss : 0.00020804045198019594
Loss : 0.00015697623894084245
Loss : 0.00012554744898807257
Loss : 0.000103945450973697
Loss : 8.81961896084249e-05
Loss : 7.647493475815281e-05
### End Training ###
### Test Set으로 Accuracy 측정 ###
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1242
           1       0.98      0.99      0.99      1429
           2       0.97      0.98      0.97      1276
           3       0.97      0.96      0.97      129